### Unsupervised Net

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm
from cv2.ximgproc import guidedFilter
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

import numpy as np
import sys

sys.path.insert(0, '..')
from utils.collection import HazeCollection
from utils.network import VaeNet, HazyNet
from utils.loss import StdLoss
from utils.utils import get_atmosphere, torch_to_np, np_to_torch, save_image, get_information

#### Collection

In [2]:
haze_path_dt0 = "/home/maldonadoq/Datasets/Reside/Standard/OWN/hazy_only"
image_path_dt0 = "/home/maldonadoq/Datasets/Reside/Standard/OWN/gt_only"

haze_path_dt1 = "/home/maldonadoq/Datasets/Reside/Standard/HSTS/synthetic/synthetic"
image_path_dt1 = "/home/maldonadoq/Datasets/Reside/Standard/HSTS/synthetic/original"

haze_path_dt2 = "/home/maldonadoq/Datasets/Reside/Standard/HSTS/real-world"

haze_path_dt3 = "/home/maldonadoq/Datasets/Reside/Standard/SOTS/indoor/hazy_val"
image_path_dt3 = "/home/maldonadoq/Datasets/Reside/Standard/SOTS/indoor/gt_val"

In [3]:
size = 256
channels = 3

collection_dt0 = HazeCollection(haze_path_dt0, image_path_dt0)
collection_dt1 = HazeCollection(haze_path_dt1, image_path_dt1)
collection_dt2 = HazeCollection(haze_path_dt2)
collection_dt3 = HazeCollection(haze_path_dt3, image_path_dt3)

#### Model

In [4]:
# UNet
un = HazyNet(channels)

x = torch.zeros([1, channels, size, size])
a = un(x)
a.shape

torch.Size([1, 3, 256, 256])

#### Training

In [5]:
class XhotTrainer:
    def __init__(
        self,
        metrics=[]
    ):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.metrics = metrics

    def init(self, image):
        self.model = {
            "netJ": HazyNet(out_channel=3).to(self.device),
            "netT": HazyNet(out_channel=1).to(self.device),
            "netA": VaeNet(image.shape).to(self.device)
        }
        self.loss = {
            "mse": nn.MSELoss().to(self.device),
            "std": StdLoss().to(self.device)
        }

        atmosphere = get_atmosphere(image)
        self.valA = nn.Parameter(data=torch.cuda.FloatTensor(
            atmosphere.reshape((1, 3, 1, 1))), requires_grad=False)

        parameters = [p for p in self.model["netJ"].parameters()]
        parameters += [p for p in self.model["netT"].parameters()]
        parameters += [p for p in self.model["netA"].parameters()]

        self.optimizer = optim.Adam(
            parameters,
            lr=self.learning_rate
        )

    def t_matting(self, mask_out_np, original):
        refine_t = guidedFilter(original.transpose(1, 2, 0).astype(
            np.float32), mask_out_np[0].astype(np.float32), 50, 1e-4)
        return np.array([np.clip(refine_t, 0.1, 1)])

    def train(self, information, epochs):
        haze, image, name = information
        self.init(haze)
        loop = tqdm(range(epochs))

        losses = []
        lossFinal = None

        hazeTorch = np_to_torch(haze)
        hazeTorch = hazeTorch.to(device=self.device)
        for i in loop:
            # forward
            predT = self.model["netT"](hazeTorch)
            predJ = self.model["netJ"](hazeTorch)
            predA = self.model["netA"](hazeTorch)

            #lossT = self.loss["std"](predT)
            lossJ = self.loss["mse"](predT * predJ + (1 - predT) * predA, hazeTorch)
            lossA = self.model["netA"].getLoss()

            lossFinal = lossJ + lossA
            lossFinal += 0.1 * self.loss["std"](predA)
            lossFinal += self.loss["mse"](predA,self.valA * torch.ones_like(predA))

            #if i % 100 == 0 or i == epochs-1:
            if i == epochs-1:
                imageJ = np.clip(torch_to_np(predJ), 0, 1)
                imageT = np.clip(torch_to_np(predT), 0, 1)
                imageA = np.clip(torch_to_np(predA), 0, 1)
                imageT = self.t_matting(imageT, haze)

                # I(x) = J(x)t(x) + A(1 − t(x))
                post = np.clip((haze - ((1 - imageT) * imageA)) / imageT, 0, 1)
                save_image('{}_{}'.format(name[0].split('.')[0], i+1), post, '../images/xhot/' + self.dt_number)
                save_image('J{}_{}'.format(name[0].split('.')[0], i+1), imageJ, '../images/xhot/' + self.dt_number)
                save_image('T{}_{}'.format(name[0].split('.')[0], i+1), imageT, '../images/xhot/' + self.dt_number)
                save_image('A{}_{}'.format(name[0].split('.')[0], i+1), imageA, '../images/xhot/' + self.dt_number)

            # backward
            self.optimizer.zero_grad()
            lossFinal.backward(retain_graph=True)
            self.optimizer.step()

            # update tqdm
            loop.set_postfix(loss=lossFinal.item())
            losses.append(lossFinal.item())
        
        # metrics
        finalSSIM, finalPSNR = 0, 0
        if image is not None:
            finalSSIM = ssim(post, image, channel_axis=0, data_range=1)
            finalPSNR = psnr(post, image)
            print("Ssim: {:.4f}, Psnr: {:.4f}\n".format(finalSSIM, finalPSNR))

        return [np.sum(losses)/len(losses), finalSSIM, finalPSNR]

    def fit(
        self,
        collection,
        epochs=1,
        dt_number='',
        learning_rate=1e-3
    ):
        historial = []
        self.dt_number = dt_number
        self.learning_rate = learning_rate
        #loop = enumerate(tqdm(collection))
        for i, info in enumerate(collection):
            print('Image {}/{} [{}]'.format(i+1, len(collection), info[2][0]))
            loss, ssim, psnr = self.train(info, epochs)
            historial.append([loss, ssim, psnr])
        return historial

In [6]:
dehazing = XhotTrainer()

#### Testing

In [7]:
epochs = 512
learning_rate = 1e-4

##### D0

In [8]:
historial_dt0 = dehazing.fit(collection_dt0, epochs, 'd0/', learning_rate)
get_information(historial_dt0, collection_dt0)

Image 1/5 [8180_1.png]


100%|██████████| 1024/1024 [00:33<00:00, 30.75it/s, loss=0.00134]


Ssim: 0.9453, Psnr: 23.8498

Image 2/5 [4561_1.png]


100%|██████████| 1024/1024 [00:30<00:00, 33.68it/s, loss=0.0107]


Ssim: 0.5049, Psnr: 15.5396

Image 3/5 [1436_5.png]


...

Ssim: 0.9216, Psnr: 24.0010

Image 4/5 [3146_1.png]


100%|██████████| 1024/1024 [00:31<00:00, 32.79it/s, loss=0.00437]


Ssim: 0.4950, Psnr: 18.9505

Image 5/5 [1401_5.png]


100%|██████████| 1024/1024 [00:55<00:00, 18.31it/s, loss=0.0101]


Ssim: 0.7483, Psnr: 18.3436

Mean SSIM: 0.7229967176914215
Mean PSNR: 20.136881178267693
Best SSIM: 8180_1.png
Best PSNR: 1436_5.png


##### D1

In [8]:
historial_dt1 = dehazing.fit(collection_dt1, epochs, 'd1/', learning_rate)
get_information(historial_dt1, collection_dt1)

Image 1/10 [1381.jpg]


100%|██████████| 512/512 [00:20<00:00, 25.03it/s, loss=0.00305]


Ssim: 0.8138, Psnr: 18.6684

Image 2/10 [5576.jpg]


100%|██████████| 512/512 [00:14<00:00, 34.34it/s, loss=0.00786]


Ssim: 0.6093, Psnr: 20.2566

Image 3/10 [7471.jpg]


...

Ssim: 0.9275, Psnr: 23.8228

Image 9/10 [1352.jpg]


100%|██████████| 512/512 [00:41<00:00, 12.30it/s, loss=0.0125]


Ssim: 0.4437, Psnr: 13.1064

Image 10/10 [4561.jpg]


100%|██████████| 512/512 [00:15<00:00, 32.96it/s, loss=0.0285]


Ssim: 0.9188, Psnr: 24.7662

Mean SSIM: 0.7667982876300812
Mean PSNR: 19.96091501998872
Best SSIM: 5920.jpg
Best PSNR: 5920.jpg


##### D2

In [9]:
historial_dt2 = dehazing.fit(collection_dt2, epochs, 'd2/', learning_rate)
get_information(historial_dt2, collection_dt2)

Image 1/10 [SFC_Google_197.jpeg]


100%|██████████| 512/512 [00:31<00:00, 16.35it/s, loss=0.0101]


Image 2/10 [MLS_Bing_117.jpeg]


100%|██████████| 512/512 [00:36<00:00, 14.04it/s, loss=0.00907]


Image 3/10 [HazyDr_Google_396.jpeg]


...

Image 9/10 [KRO_Google_143.jpeg]


100%|██████████| 512/512 [00:36<00:00, 14.10it/s, loss=0.0262]


Image 10/10 [HazeDr_Google_404.jpeg]


100%|██████████| 512/512 [01:32<00:00,  5.56it/s, loss=0.0577]

Mean SSIM: 0.0
Mean PSNR: 0.0
Best SSIM: SFC_Google_197.jpeg
Best PSNR: SFC_Google_197.jpeg


##### D3

In [8]:
historial_dt3 = dehazing.fit(collection_dt3, epochs, 'd3/', learning_rate)
get_information(historial_dt3, collection_dt3)

Image 1/10 [1410_5.png]


100%|██████████| 512/512 [00:27<00:00, 18.75it/s, loss=0.00529]


Ssim: 0.7931, Psnr: 18.9389

Image 2/10 [1430_5.png]


100%|██████████| 512/512 [00:26<00:00, 18.97it/s, loss=0.00649]


Ssim: 0.7234, Psnr: 20.6365

Image 3/10 [1440_5.png]


...

Ssim: 0.8580, Psnr: 19.7209

Image 9/10 [1425_5.png]


100%|██████████| 512/512 [00:27<00:00, 18.37it/s, loss=0.00989]


Ssim: 0.6793, Psnr: 16.1226

Image 10/10 [1420_5.png]


100%|██████████| 512/512 [00:27<00:00, 18.69it/s, loss=0.00535]

Ssim: 0.8557, Psnr: 18.2756

Mean SSIM: 0.7946922481060028
Mean PSNR: 18.747153025619387
Best SSIM: 1415_5.png
Best PSNR: 1430_5.png
